In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pulp import *

In [59]:
inputs = pd.read_csv('inputs.csv', index_col=0, delimiter=';')
outputs = pd.read_csv('outputs.csv', index_col=0, delimiter=';')
samples = pd.read_csv('samples_homework.csv', delimiter=';')

In [60]:
efficiency_scores = {}
reference_units = {}
adjustments = {}
super_efficiency_scores = {}
cross_efficiency_scores = {}
expected_efficiency = {}

In [63]:
for DMUo in inputs.index:
    problem = LpProblem(f"Efficiency_{DMUo}", LpMinimize)
    lambdas = LpVariable.dicts("Lambda", (i for i in inputs.index), lowBound=0)
    theta = LpVariable("theta", lowBound=0, upBound=1)

    problem += theta

    for input_metric in inputs.columns:
        problem += lpSum([lambdas[i] * inputs.loc[i, input_metric] for i in inputs.index]) <= theta * inputs.loc[DMUo, input_metric]

    for output_metric in outputs.columns:
        problem += lpSum([lambdas[i] * outputs.loc[i, output_metric] for i in outputs.index]) >= outputs.loc[DMUo, output_metric]

    problem.solve(PULP_CBC_CMD(msg=False))

    if LpStatus[problem.status] == 'Optimal':
        efficiency_score = pulp.value(theta)
        efficiency_scores[DMUo] = efficiency_score

        if efficiency_score < 1:
            reference_units[DMUo] = {}
            adjustments[DMUo] = {}
            for i in inputs.index:
                if lambdas[i].varValue > 0:
                    reference_units[DMUo][i] = lambdas[i].varValue

            for input_metric in inputs.columns:
                optimal_input = sum(lambdas[i].varValue * inputs.loc[i, input_metric] for i in inputs.index)
                adjustments[DMUo][input_metric] = optimal_input

    else:
        print(f"Problem for {DMUo} is not solvable")
    
    # Superefektywność
    super_problem = LpProblem(f"SuperEfficiency_{DMUo}", LpMinimize)
    super_lambdas = LpVariable.dicts("SuperLambda", (i for i in inputs.index if i != DMUo), lowBound=0)
    super_theta = LpVariable("super_theta", lowBound=0)

    super_problem += super_theta
    for input_metric in inputs.columns:
        super_problem += lpSum([super_lambdas[i] * inputs.loc[i, input_metric] for i in inputs.index if i != DMUo]) <= super_theta * inputs.loc[DMUo, input_metric]
    for output_metric in outputs.columns:
        super_problem += lpSum([super_lambdas[i] * outputs.loc[i, output_metric] for i in outputs.index if i != DMUo]) >= outputs.loc[DMUo, output_metric]
    
    super_problem.solve(PULP_CBC_CMD(msg=False))
    if LpStatus[super_problem.status] == 'Optimal':
        super_efficiency_scores[DMUo] = 1 / pulp.value(super_theta)

# Obliczanie średniej efektywności krzyżowej i oczekiwanej wartości
for DMUo in inputs.index:
    cross_efficiency_scores[DMUo] = []
    for index, sample in samples.iterrows():
        sample_efficiency = []
        for input_metric, output_metric in zip(inputs.columns, outputs.columns):
            problem = LpProblem("SampleEfficiency", LpMinimize)
            lambdas = LpVariable.dicts("Lambda", (i for i in inputs.index), lowBound=0)
            theta = LpVariable("theta", lowBound=0, upBound=1)
            
            problem += theta
            problem += lpSum([lambdas[i] * inputs.loc[i, input_metric] * sample[input_metric] for i in inputs.index]) <= theta * inputs.loc[DMUo, input_metric] * sample[input_metric]
            problem += lpSum([lambdas[i] * outputs.loc[i, output_metric] * sample[output_metric] for i in outputs.index]) >= outputs.loc[DMUo, output_metric] * sample[output_metric]
            
            problem.solve(PULP_CBC_CMD(msg=False))
            if LpStatus[problem.status] == 'Optimal':
                sample_efficiency.append(pulp.value(theta))

        cross_efficiency_scores[DMUo].append(np.mean(sample_efficiency))
    expected_efficiency[DMUo] = np.mean(cross_efficiency_scores[DMUo])


for airport, efficiency in efficiency_scores.items():
    print(f"Efficiency for {airport}: {efficiency:.2f}")
    if airport in super_efficiency_scores:
        print(f"Super Efficiency for {airport}: {super_efficiency_scores[airport]:.2f}")
    if airport in adjustments:
        print(f"Adjustments for {airport}:")
        for metric, value in adjustments[airport].items():
            print(f"  {metric}: reduce to {value:.2f}")
    print(f"Expected Efficiency for {airport}: {expected_efficiency[airport]:.2f}")

Efficiency for WAW: 1.00
Super Efficiency for WAW: 0.44
Expected Efficiency for WAW: 0.73
Efficiency for KRK: 1.00
Super Efficiency for KRK: 0.89
Expected Efficiency for KRK: 0.46
Efficiency for KAT: 0.59
Super Efficiency for KAT: 1.69
Adjustments for KAT:
  i1: reduce to 2.13
  i2: reduce to 18.92
  i3: reduce to 33.94
  i4: reduce to 4.40
Expected Efficiency for KAT: 0.26
Efficiency for WRO: 1.00
Super Efficiency for WRO: 0.96
Expected Efficiency for WRO: 0.47
Efficiency for POZ: 0.80
Super Efficiency for POZ: 1.25
Adjustments for POZ:
  i1: reduce to 1.20
  i2: reduce to 8.00
  i3: reduce to 19.20
  i4: reduce to 1.93
Expected Efficiency for POZ: 0.44
Efficiency for LCJ: 0.30
Super Efficiency for LCJ: 3.33
Adjustments for LCJ:
  i1: reduce to 0.18
  i2: reduce to 2.78
  i3: reduce to 7.20
  i4: reduce to 0.47
Expected Efficiency for LCJ: 0.17
Efficiency for GDN: 1.00
Super Efficiency for GDN: 0.50
Expected Efficiency for GDN: 0.72
Efficiency for SZZ: 0.27
Super Efficiency for SZZ: 3